In [ ]:
import requests

from dotenv import load_dotenv
import os

load_dotenv()

hugging_face_api_key = os.getenv('HUGGING_FACE_API_KEY')

API_URL = "https://api-inference.huggingface.co/models/jonatasgrosman/wav2vec2-large-xlsr-53-english"
headers = {"Authorization": "Bearer "}
headers["Authorization"] += hugging_face_api_key

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query("sample.flac")

print(output)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import joblib

# Load Model
pipe_lr = joblib.load(open("./models/emotion_classifier_pipe_lr.pkl", "rb"))

# Function
def predict_emotions(docx):
    results = pipe_lr.predict([docx])
    return results[0]

def get_prediction_proba(docx):
    results = pipe_lr.predict_proba([docx])
    return results

emotions_emoji_dict = {"anger": "😠", "disgust": "🤮", "fear": "😨😱", "happy": "🤗", "joy": "😂", "neutral": "😐", "sad": "😔", "sadness": "😔", "shame": "😳", "surprise": "😮"}

# Main Application
def main():
    print("Welcome to Emotion Detection in Text")
    raw_text = input("Type Here: ")
    
    prediction = predict_emotions(raw_text)
    probability = get_prediction_proba(raw_text)
    
    
    print("\nOriginal Text:")
    print(raw_text)
    print("\nPrediction:")
    emoji_icon = emotions_emoji_dict[prediction]
    print("{}:{}".format(prediction, emoji_icon))
    print("Confidence:{}".format(np.max(probability)))
    
    print("\nPrediction Probability:")
    proba_df = pd.DataFrame(probability, columns=pipe_lr.classes_)
    proba_df_clean = proba_df.T.reset_index()
    proba_df_clean.columns = ["emotions", "probability"]
    print(proba_df_clean)

if __name__ == '__main__':
    main()